In [ ]:
from colabtools import adhoc_import
from pyshp import shapefile as shp  # Requires the pyshp package
from descartes import PolygonPatch
import matplotlib.pyplot as plt

### Initial shapefiles (authorized / not authorized)

* Sparse
* Provided by Imaflora via Alessandro Germano

Definitions:
* autorizada: Legal to chop down trees
* não autorizada: Illegal to chop down trees
* \<uncovered area\>: ???

In [ ]:
def plot_format1():
  # TODO: Replace with amazon shapefil in gdrive
  sf = shp.Reader("/usr/local/google/home/nicholasroth/simex_2021/te_amz_exploracao_simex_2021.shp")

  fig = plt.figure(figsize=(50,50))
  ax = fig.gca()
  BLUE = '#6699cc'
  RED = '#cc6666'

  for shape in sf.shapeRecords():
    category = shape.record.as_dict()['categoria']
    #'não autorizada' vs 'autorizada'
    if category == 'autorizada':
      ax.add_patch(PolygonPatch(shape.shape.__geo_interface__, fc=BLUE, ec=BLUE, alpha=1, zorder=2 ))
    elif category == 'não autorizada':
      ax.add_patch(PolygonPatch(shape.shape.__geo_interface__, fc=RED, ec=RED, alpha=1, zorder=2 ))
    else:
      raise ValueError("Unexpected category")

  ax.axis('scaled')
  plt.show()

plot_format1()

### V2 Shapefiles (nuanced)

Definitions:
* **PMFS (Amazônia Legal)**: Data from requests/authorizations for exploration licensing of the Sustainable Forest Management Plan (PMFS) type registered in the National System for Control of the Origin of Forest Products (Sinaflor) and located in the Legal Amazon region.
  * Legal to log
* **PMFS  (Outros Biomas)**: Data from requests/authorizations for exploration licensing of the Sustainable Forest Management Plan (PMFS) type registered in the National System for the Control of the Origin of Forest Products (Sinaflor) and located outside the Legal Amazon.
  * Legal to log
* **Autorização de Supressão de Vegetação - ASV**: Data from requests/authorizations for exploitation licensing of the Authorization for Vegetation Suppression (ASV) registered in the National System for Control of the Origin of Forest Products (Sinaflor).
  * Legal to log
* **Uso Alternativo do Solo**: Data from requests/authorizations for exploration licensing of the Alternative Soil Use (UAS) type registered in the National System for Control of the Origin of Forest Products (Sinaflor).
  * Legal to log
* \<uncovered area\>: Unknown
  * Illegal to log


Questions for Alessandro:
* Do these definitions match your expectations?
* Do my interpretations in sub-bullets match your expectations?

Alessandro (2023-02-13): I believe you are right regarding PMFS, ASV and Uso Alternativo, but will confirm with Imaflora. Regarding uncovered areas, you should treat them as areas of non-permitted exploration—uncovered areas in the map means that there is no paper trail saying timber can be harvested there.


In [ ]:
def get_shp_categories(path: str, encoding: str = 'utf8'):
  sf = shp.Reader(path, encoding=encoding)
  #categories = set()
  nom_ativides = set()
  tipo_areas = set()

  for shape in sf.shapeRecords():
    #print(shape.record.as_dict())
    nom_ativid = shape.record.as_dict()['nom_ativid']
    tipo_area = shape.record.as_dict()['tipo_area']
    nom_ativides.add(nom_ativid)
    tipo_areas.add(tipo_area)

  return list(nom_ativides), list(tipo_areas)

In [ ]:
def plot_format2():

  sf = shp.Reader("/usr/local/google/home/nicholasroth/vw_upa_sinaflor_a/vw_upa_sinaflor_a.shp", encoding="ISO8859-1")

  BLUE = '#6699cc'
  GREEN = "#29b335"
  YELLOW = "#ffff2e"
  ORANGE = '#f58f14'
  BROWN = "#b37929"
  colors = {
      'PMFS (Amazônia Legal)': GREEN,
      'PMFS  (Outros Biomas)': YELLOW,
      'Autorização de Supressão de Vegetação - ASV': ORANGE,
      'Uso Alternativo do Solo': BROWN,
  }
  fig = plt.figure(figsize=(50,50))
  ax = fig.gca()

  for shape in sf.shapeRecords():
    category = shape.record.as_dict()['nom_ativid']
    if category in colors.keys():
      ax.add_patch(PolygonPatch(shape.shape.__geo_interface__, fc=colors[category], ec=colors[category], alpha=1, zorder=2 ))
    else:
      print(category)
      raise ValueError("Unexpected category")

  ax.axis('scaled')
  plt.show()

plot_format2()